In [1]:
import colorsys
import cv2
import numpy as np
import csv
import matplotlib.pyplot as plt
import os
import pandas as pd
os.environ["OMP_NUM_THREADS"] = "4"
path = './dataset/test_images/image'  # Modify this according to your project structure
#print("Current Working Directory:", os.getcwd())
#print("Full path to check:", os.path.abspath(path))

In [2]:
def get_color_frequency(image):
    # Get the image size
    height, width, _ = image.shape
    # Initialize an empty dictionary to store color frequencies
    color_frequency = {}
    # Loop through each pixel in the image
    for y in range(height):
        for x in range(width):
            # Get the RGB values of the pixel
            r, g, b = image[y, x]
            # Create a tuple representing the color
            color = (r, g, b)
            # Increment the frequency if the color is already in the dictionary, else insert a new row
            if color in color_frequency:
                color_frequency[color] += 1
            else:
                color_frequency[color] = 1
    return color_frequency


def rgb_to_hsv(rgb):
    r, g, b = [x / 255.0 for x in rgb]
    return colorsys.rgb_to_hsv(r, g, b)

def classify_color(rgb):
    hsv = rgb_to_hsv(rgb)
    h, s, v = hsv
    h *= 360
    color_ranges = [
        (0, 15, "black"), (345, 360, "black"),(0, 15, "white"),(345, 360, "white"),
        (15, 45, "Red"),(45, 75, "Orange"),
        (75, 165, [
            ("Inchworm", 90),("Lawn Green", 120),("Bright Green", 135),
            ("Celadon", 150),("Pastel Green", 165),("Green", 165)
        ]),
        (165, 255, [
            ("Pistachio", 180),("Dollar Bill", 195),("Asparagus", 210), ("Dark Pastel Green", 225),
            ("Camouflage Green", 240), ("India Green", 255), ("Green", 255)
        ]),
        (255, 285, [
            ("Blue", 270),("Indigo", 285),("Purple", 285)
        ]),
        (285, 345, [
            ("Pink", 300),("Dark Olive Green", 315),("Rifle Green", 330), ("Green", 345)
        ]),
        (345, 360, "RAINBOW")
    ]

    for start, end, color in color_ranges:
        if start <= h < end:
            if isinstance(color, list):
                for sub_color, sub_end in color:
                    if h < sub_end:
                        return sub_color
            else:
                return color
    return "Unknown"

def divide_image(image):
    height, width, channels = image.shape
    grid_size = 32
    print(height,width)
    rows = height // grid_size
    cols = width // grid_size
    for i in range(rows):
        for j in range(cols):
            start_row = i * grid_size
            end_row = start_row + grid_size
            start_col = j * grid_size
            end_col = start_col + grid_size
            # extract 32x32 pixel grid from the image
            grid = image[start_row:end_row, start_col:end_col]
                # apply a Gaussian filter to the grid
                #sigma = 1.5
                # filtered_grid = gaussian_filter(grid, sigma=sigma)
            cv2.imwrite(path+'/vegetation/'+f'grid_{i}_{j}.png', grid)
                
def write_output_to_csv(image, path):
    height, width, channels = image.shape
    
    grid_size = 32
    rows = height // grid_size
    cols = width // grid_size
    csv_file_path = path +'/data.csv'

    with open(csv_file_path, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Write the header row
        header = ["grid", "color1_h","color1_s","color1_v", "frequency", "colour_name",
                  "color2_h","color2_s","color2_v", "frequency", "colour_name",
                  "color3_h","color3_s","color3_v", "frequency", "colour_name",
                  "color4_h","color4_s","color4_v", "frequency", "colour_name"]
        csv_writer.writerow(header)

        for i in range(rows):
            for j in range(cols):
                start_row = i * grid_size
                end_row = start_row + grid_size
                start_col = j * grid_size
                end_col = start_col + grid_size
                # extract 32x32 pixel grid from the image
                grid = image[start_row:end_row, start_col:end_col]
                color_frequency = get_color_frequency(grid)

                # sort the colors by frequency in descending order
                sorted_colors = sorted(color_frequency.items(), key=lambda x: x[1], reverse=True)
                csv_row = [f"Grid({i}, {j})"]

                # print the grid indices and the 4 dominant colors with their frequencies
                #print(f"Grid({i}, {j}): Dominant Colors and Frequencies:")
                for idx in range(min(4, len(sorted_colors))):
                    color, frequency = sorted_colors[idx]
                    color_name = classify_color(color)
                    #print(f"   Color {idx + 1}: RGB {color} - Frequency: {frequency} - Color Name: {color_name}")
                    csv_row.extend([color[0], color[1], color[2], frequency, color_name])

                # Fill in the missing columns if there are fewer than 4 colors
                while len(csv_row) < len(header):
                    csv_row.extend(["", "", "", "", ""])

                # Write the CSV row to the file
                csv_writer.writerow(csv_row)


In [3]:
image_path = os.path.join(path, 'image.jpg')  # Dynamically join the path correctly
print(f"Attempting to read image from: {image_path}")
imge = cv2.imread(image_path)
m,n,r=imge.shape
seg_frame=np.zeros([m,n,r])
b,g,r=cv2.split(imge)
img=cv2.merge((r,g,b))
divide_image(img)
write_output_to_csv(img, path)


Attempting to read image from: ./dataset/test_images/image\image.jpg
343 680


In [4]:
import pandas as pd
import os

def create_new_csv(input_csv):
    # Read the input csv file
    df = pd.read_csv(input_csv)
    
    # Select the required columns only
    required_columns = ['grid', 'color1_h', 'color1_s', 'color1_v', 'color2_h', 'color2_s', 'color2_v', 'color3_h', 'color3_s', 'color3_v', 'color4_h', 'color4_s', 'color4_v']
    df_required = df[required_columns]
    
    # Save the new csv file with the desired header
    new_csv = 'validation.csv'  # Specify the name of the new csv file
    new_csv_path = './dataset/validation/'  # Construct the full path of the new csv file
    df_required.to_csv(os.path.join(new_csv_path, new_csv), index=False)
    print("validation.csv created in new directory")

csv_path ='./dataset/test_images/image/data.csv'
create_new_csv(csv_path)

validation.csv created in new directory


In [5]:
import pandas as pd
import colorsys

# Read input CSV file
input_file = "./dataset/test_images/image/data.csv"
df_input = pd.read_csv(input_file)

# Function to convert HSV to RGB
def hsv_to_rgb(h, s, v):
    r, g, b = colorsys.hsv_to_rgb(h / 360.0, s / 100, v / 100)
    return round(b * 255), round(g * 255), round(r * 255)

# Process data and generate output
output_data = []

for index, row in df_input.iterrows():
    grid = row['grid']
    
    # Color 1
    h1, s1, v1 = row['color1_h'], row['color1_s'], row['color1_v']
    #rgb1 = hsv_to_rgb(h1, s1, v1)
    color1_name = row['colour_name']
    
    # Color 2
    h2, s2, v2 = row['color2_h'], row['color2_s'], row['color2_v']
    #rgb2 = hsv_to_rgb(h2, s2, v2)
    color2_name = row['colour_name.1']
    
    # Color 3
    h3, s3, v3 = row['color3_h'], row['color3_s'], row['color3_v']
    #rgb3 = hsv_to_rgb(h3, s3, v3)
    color3_name = row['colour_name.2']
    
    # Color 4
    h4, s4, v4 = row['color4_h'], row['color4_s'], row['color4_v']
    #rgb4 = hsv_to_rgb(h4, s4, v4)
    color4_name = row['colour_name.3']
    
    output_data.append([grid, h1,s1,v1, color1_name, h2,s2,v2, color2_name,h3,s3,v3, color3_name, h4,s4,v4, color4_name])

# Create a new DataFrame for the output
columns = ['grid', 'color1_h', 'color1_s', 'color1_v', 'colour_name',
           'color2_h', 'color2_s', 'color2_v', 'colour_name.1',
           'color3_h', 'color3_s', 'color3_v', 'colour_name.2',
           'color4_h', 'color4_s', 'color4_v', 'colour_name.3']
output_df = pd.DataFrame(output_data, columns=columns)

# Identify repeated colors
color_columns = ['colour_name', 'colour_name.1', 'colour_name.2', 'colour_name.3']
output_df['repeated_colour'] = output_df[color_columns].apply(lambda row: row.mode()[0] if row.value_counts().max() >= 3 else '', axis=1)

# Filter rows with non-empty 'repeated_colour' values
output_df_filtered = output_df[output_df['repeated_colour'] != '']

# Write the output CSV file
output_file = "./dataset/test_images/image/repeated_colour.csv"
output_df_filtered.to_csv(output_file, index=False)


In [6]:
import pandas as pd
df=pd.read_csv("./dataset/test_images/image/repeated_colour.csv")
color_freq={}
for color in df['repeated_colour']:
    if color in color_freq:
        color_freq[color]+=1
    else:
        color_freq[color]=1
        
print(color_freq)

{'Dark Pastel Green': 115, 'Camouflage Green': 35, 'Inchworm': 6, 'Orange': 9, 'India Green': 20, 'Blue': 1, 'Lawn Green': 1}


In [7]:
import pandas as pd

# Step 1: Load the CSV file
csv_file_path = './dataset/test_images/image/repeated_colour.csv'  # Replace with the actual file path
df = pd.read_csv(csv_file_path)

# Step 2: Drop the specified columns
columns_to_drop = ['colour_name', 'colour_name.1', 'colour_name.2', 'colour_name.3']
df = df.drop(columns=columns_to_drop)

# Step 3: Save the modified DataFrame to a new CSV file
new_csv_path = './dataset/test_images/image/test.csv'  # Replace with where you want to save the new CSV file
df.to_csv(new_csv_path, index=False)

print("CSV file saved without the specified columns.")


CSV file saved without the specified columns.
